In [4]:
import csv
from pathlib import Path
import numpy as np
import pandas as pd
import time
from enum import Enum

def get_all_subject_ids():#번호 다 가져오는 함수
    subjects_as_ints = [3509524, 5132496, 1066528, 5498603, 2638030, 2598705, 5383425, 1455390, 4018081, 9961348,1449548, 8258170, 781756, 9106476, 8686948, 8530312, 3997827, 4314139, 1818471, 4426783, 8173033, 7749105, 5797046, 759667, 8000685, 6220552, 844359, 9618981, 1360686, 46343, 8692923]
    subjects_as_strings = []
    for subject in subjects_as_ints:
        subjects_as_strings.append(str(subject))
    return subjects_as_strings

def get_project_root() -> Path:
    return Path()

def get_intersecting_interval(collection_list):
    start_times = []
    end_times = []
    for collection in collection_list:
        interval = collection.get_interval()
        start_times.append(interval.start_time)
        end_times.append(interval.end_time)
    return Interval(start_time=max(start_times), end_time=min(end_times))

def read_precleaned(subject_id):
    psg_path = str(Path().joinpath('labels/' + subject_id + '_labeled_sleep.txt'))
    data = []

    with open(psg_path, 'rt') as csv_file:
        file_reader = csv.reader(csv_file, delimiter=' ', quotechar='|')
        count = 0
        rows_per_epoch = 1
        for row in file_reader:
            if count == 0:
                start_time = float(row[0])
                start_score = int(row[1])
                epoch = Epoch(timestamp=start_time, index=1)
                data.append(StageItem(epoch=epoch, stage=PSGConverter.get_label_from_int(start_score)))
            else:
                timestamp = start_time + count * 30
                score = int(row[1])
                epoch = Epoch(timestamp=timestamp,index=(1 + int(np.floor(count / rows_per_epoch))))

                data.append(StageItem(epoch=epoch, stage=PSGConverter.get_label_from_int(score)))
            count = count + 1
    return PSGRawDataCollection(subject_id=subject_id, data=data)  

def remove_repeats(array):
    array_no_repeats = np.unique(array, axis=0)
    array_no_repeats = array_no_repeats[np.argsort(array_no_repeats[:, 0])]
    return array_no_repeats

def load(motion_file, delimiter=' '):
    motion_array = pd.read_csv(str(motion_file), delimiter=delimiter).values
    return motion_array

def get_raw_file_path(subject_id):
    project_root = get_project_root()
    return project_root.joinpath('motion/' + subject_id + '_acceleration.txt')

def load_raw(subject_id):
    raw_motion_path = get_raw_file_path(subject_id)
    motion_array = load(raw_motion_path)
    motion_array = remove_repeats(motion_array)
    return MotionCollection(subject_id=subject_id, data=motion_array)

def crop_all(subject_id):
    psg_raw_collection = read_precleaned(subject_id)  # Loads already extracted PSG data
    motion_collection = load_raw(subject_id)
    
    valid_interval = get_intersecting_interval([psg_raw_collection, motion_collection])
    psg_raw_collection = PSGService.crop(psg_raw_collection, valid_interval)
    motion_collection = MotionService.crop(motion_collection, valid_interval)
    PSGService.write(psg_raw_collection)
    MotionService.write(motion_collection)
    #ActivityCountService.build_activity_counts_without_matlab(subject_id, motion_collection.data)  # Builds activity counts with python, not MATLAB

In [5]:
class Interval(object):
    def __init__(self, start_time, end_time):
        self.start_time = start_time
        self.end_time = end_time

In [6]:
class MotionCollection(object):
    def __init__(self, subject_id, data):
        self.subject_id = subject_id
        self.data = data
        self.timestamps = data[:, 0]
        self.values = data[:, 1:]

    def get_interval(self):
        return Interval(start_time=np.amin(self.data[:, 0]),
                        end_time=np.amax(self.data[:, 0]))

In [8]:
class SleepStage(Enum):
    wake = 0
    n1 = 1
    n2 = 2
    n3 = 3
    n4 = 4
    rem = 5
    unscored = -1

In [9]:
class PSGRawDataCollection(object):
    def __init__(self, subject_id, data: [SleepStage]):
        self.subject_id = subject_id
        self.data = data

    def get_np_array(self):
        number_of_epochs = len(self.data)
        array = np.zeros((number_of_epochs, 2))
        for index in range(number_of_epochs):
            stage_item = self.data[index]
            array[index, 0] = stage_item.epoch.timestamp
            array[index, 1] = stage_item.stage.value
        return array

    def get_interval(self):
        number_of_epochs = len(self.data)
        min_timestamp = 1e15
        max_timestamp = -1
        for index in range(number_of_epochs):
            stage_item = self.data[index]
            if stage_item.epoch.timestamp < min_timestamp:
                min_timestamp = stage_item.epoch.timestamp
            if stage_item.epoch.timestamp > max_timestamp:
                max_timestamp = stage_item.epoch.timestamp
        return Interval(start_time=min_timestamp, end_time=max_timestamp)

In [10]:
class StageItem(object):
    def __init__(self, epoch, stage: SleepStage):
        self.epoch = epoch
        self.stage = stage

In [11]:
class PSGConverter(object):
    strings_to_labels = {
        "?": SleepStage.unscored,
        "W": SleepStage.wake,
        "1": SleepStage.n1,
        "N1": SleepStage.n1,
        "2": SleepStage.n2,
        "N2": SleepStage.n2,
        "3": SleepStage.n3,
        "N3": SleepStage.n3,
        "4": SleepStage.n4,
        "N4": SleepStage.n4,
        "R": SleepStage.rem,
        "M": SleepStage.wake}

    ints_to_labels = {
        -1: SleepStage.unscored,
        0: SleepStage.wake,
        1: SleepStage.n1,
        2: SleepStage.n2,
        3: SleepStage.n3,
        4: SleepStage.n4,
        5: SleepStage.rem,
        6: SleepStage.unscored}

    @staticmethod
    def get_label_from_string(stage_string):
        if stage_string in PSGConverter.strings_to_labels:
            return PSGConverter.strings_to_labels[stage_string]

    @staticmethod
    def get_label_from_int(stage_int):
        if stage_int in PSGConverter.ints_to_labels:
            return PSGConverter.ints_to_labels[stage_int]

In [12]:
class Constants(object):
    # WAKE_THRESHOLD = 0.3  # These values were used for scikit-learn 0.20.3, See:
    # REM_THRESHOLD = 0.35  # https://scikit-learn.org/stable/whats_new.html#version-0-21-0
    WAKE_THRESHOLD = 0.5  #
    REM_THRESHOLD = 0.35

    EPOCH_DURATION_IN_SECONDS = 30
    SECONDS_PER_MINUTE = 60
    SECONDS_PER_DAY = 3600 * 24
    SECONDS_PER_HOUR = 3600
    VERBOSE = False
    CROPPED_FILE_PATH = get_project_root().joinpath('outputs/cropped/')
    FEATURE_FILE_PATH = get_project_root().joinpath('outputs/features/')
    FIGURE_FILE_PATH = get_project_root().joinpath('outputs/figures/')
    LOWER_BOUND = -0.2
    #MATLAB_PATH = '/Applications/MATLAB_R2019a.app/bin/matlab'  #Replace with your MATLAB path

In [13]:
class Epoch(object):
    DURATION = 30  # seconds
    def __init__(self, timestamp, index):
        self.timestamp = timestamp
        self.index = index

In [14]:
class PSGService(object):
    @staticmethod
    def crop(psg_raw_collection, interval):
        subject_id = psg_raw_collection.subject_id

        stage_items = []
        for stage_item in psg_raw_collection.data:
            timestamp = stage_item.epoch.timestamp
            if interval.start_time <= timestamp < interval.end_time:
                stage_items.append(stage_item)

        return PSGRawDataCollection(subject_id=subject_id, data=stage_items)
    @staticmethod
    def write(psg_raw_data_collection):
        data_array = []
        for index in range(len(psg_raw_data_collection.data)):
            stage_item = psg_raw_data_collection.data[index]
            data_array.append([stage_item.epoch.timestamp, stage_item.stage.value])
        np_psg_array = np.array(data_array)
        psg_output_path = Constants.CROPPED_FILE_PATH.joinpath(psg_raw_data_collection.subject_id + "_cleaned_psg.out")
        np.savetxt(psg_output_path, np_psg_array, fmt='%f')
    @staticmethod
    def load_cropped_array(subject_id):
        cropped_psg_path = Constants.CROPPED_FILE_PATH.joinpath(subject_id + "_cleaned_psg.out")
        return pd.read_csv(str(cropped_psg_path), delimiter=' ').values
    @staticmethod
    def load_cropped(subject_id):
        cropped_array = PSGService.load_cropped_array(subject_id)
        stage_items = []

        for row in range(np.shape(cropped_array)[0]):
            value = cropped_array[row, 1]
            stage_items.append(StageItem(epoch=Epoch(timestamp=cropped_array[row, 0], index=row),
                                         stage=PSGConverter.get_label_from_int(value)))

        return PSGRawDataCollection(subject_id=subject_id, data=stage_items)

In [15]:
class MotionService(object):
    @staticmethod
    def load(motion_file, delimiter=' '):
        motion_array = pd.read_csv(str(motion_file), delimiter=delimiter).values
        return motion_array
    @staticmethod
    def load_cropped(subject_id):
        cropped_motion_path = MotionService.get_cropped_file_path(subject_id)
        motion_array = MotionService.load(cropped_motion_path)
        return MotionCollection(subject_id=subject_id, data=motion_array)
    @staticmethod
    def crop(motion_collection, interval):
        subject_id = motion_collection.subject_id
        timestamps = motion_collection.timestamps
        valid_indices = ((timestamps >= interval.start_time)
                         & (timestamps < interval.end_time)).nonzero()[0]

        cropped_data = motion_collection.data[valid_indices, :]
        return MotionCollection(subject_id=subject_id, data=cropped_data)
    @staticmethod
    def write(motion_collection):
        motion_output_path = MotionService.get_cropped_file_path(motion_collection.subject_id)
        np.savetxt(motion_output_path, motion_collection.data, fmt='%f')
    @staticmethod
    def get_cropped_file_path(subject_id):
        return Constants.CROPPED_FILE_PATH.joinpath(subject_id + "_cleaned_motion.out")

In [16]:
class RawDataProcessor:
    BASE_FILE_PATH = get_project_root().joinpath('outputs/cropped/')

    @staticmethod
    def get_valid_epochs(subject_id):
        psg_collection = PSGService.load_cropped(subject_id)
        motion_collection = MotionService.load_cropped(subject_id)
        
        start_time = psg_collection.data[0].epoch.timestamp
        motion_epoch_dictionary = RawDataProcessor.get_valid_epoch_dictionary(motion_collection.timestamps, start_time)
        valid_epochs = []
        for stage_item in psg_collection.data:
            epoch = stage_item.epoch
            if epoch.timestamp in motion_epoch_dictionary and stage_item.stage != SleepStage.unscored:
                valid_epochs.append(epoch)
        return valid_epochs

    @staticmethod
    def get_valid_epoch_dictionary(timestamps, start_time):
        epoch_dictionary = {}

        for ind in range(np.shape(timestamps)[0]):
            time = timestamps[ind]
            floored_timestamp = time - np.mod(time - start_time, Epoch.DURATION)

            epoch_dictionary[floored_timestamp] = True

        return epoch_dictionary



In [17]:
class PSGLabelService(object):
    @staticmethod
    def load(subject_id):
        psg_label_path = PSGLabelService.get_path(subject_id)
        feature = pd.read_csv(str(psg_label_path)).values
        return feature

    @staticmethod
    def get_path(subject_id):
        return Constants.FEATURE_FILE_PATH.joinpath(subject_id + '_psg_labels.out')

    @staticmethod
    def build(subject_id, valid_epochs):
        psg_array = PSGService.load_cropped_array(subject_id)
        labels = []
        for epoch in valid_epochs:
            value = np.interp(epoch.timestamp, psg_array[:, 0], psg_array[:, 1])
            labels.append(value)
        return np.array(labels)

    @staticmethod
    def write(subject_id, labels):
        psg_labels_path = PSGLabelService.get_path(subject_id)
        np.savetxt(psg_labels_path, labels, fmt='%f')

In [18]:
class TimeBasedFeatureService(object):
    @staticmethod
    def load_time(subject_id):
        feature_path = TimeBasedFeatureService.get_path_for_time(subject_id)
        feature = pd.read_csv(str(feature_path)).values
        return feature

    @staticmethod
    def get_path_for_time(subject_id):
        return Constants.FEATURE_FILE_PATH.joinpath(subject_id + '_time_feature.out')

    @staticmethod
    def write_time(subject_id, feature):
        feature_path = TimeBasedFeatureService.get_path_for_time(subject_id)
        np.savetxt(feature_path, feature, fmt='%f')

    @staticmethod
    def load_circadian_model(subject_id):
        feature_path = TimeBasedFeatureService.get_path_for_circadian_model(subject_id)
        feature = pd.read_csv(str(feature_path), delimiter=' ').values
        return feature

    @staticmethod
    def get_path_for_circadian_model(subject_id):
        return Constants.FEATURE_FILE_PATH.joinpath(subject_id + '_circadian_feature.out')

    @staticmethod
    def write_circadian_model(subject_id, feature):
        feature_path = TimeBasedFeatureService.get_path_for_circadian_model(subject_id)
        np.savetxt(feature_path, feature, fmt='%f')

    @staticmethod
    def load_cosine(subject_id):
        feature_path = TimeBasedFeatureService.get_path_for_cosine(subject_id)
        feature = pd.read_csv(str(feature_path)).values
        return feature

    @staticmethod
    def get_path_for_cosine(subject_id):
        return Constants.FEATURE_FILE_PATH.joinpath(subject_id + '_cosine_feature.out')

    @staticmethod
    def write_cosine(subject_id, feature):
        feature_path = TimeBasedFeatureService.get_path_for_cosine(subject_id)
        np.savetxt(feature_path, feature, fmt='%f')

    @staticmethod
    def build_time(valid_epochs):
        features = []
        first_timestamp = valid_epochs[0].timestamp
        for epoch in valid_epochs:
            value = epoch.timestamp - first_timestamp

            value = value / 3600.0  # Changing units to hours improves performance

            features.append(value)
        return np.array(features)

    @staticmethod
    def build_circadian_model(subject_id, valid_epochs):
        circadian_file = utils.get_project_root().joinpath('data/circadian_predictions/' + subject_id +
                                                           '_clock_proxy.txt')
        if circadian_file.is_file():
            circadian_model = pd.read_csv(str(circadian_file), delimiter=',').values

            return TimeBasedFeatureService.build_circadian_model_from_raw(circadian_model, valid_epochs)

    @staticmethod
    def cosine_proxy(time):
        sleep_drive_cosine_shift = 5
        return -1 * np.math.cos((time - sleep_drive_cosine_shift * Constants.SECONDS_PER_HOUR) *
                                2 * np.math.pi / Constants.SECONDS_PER_DAY)

    @staticmethod
    def build_cosine(valid_epochs):
        features = []
        first_value = TimeBasedFeatureService.cosine_proxy(0)
        first_timestamp = valid_epochs[0].timestamp

        for epoch in valid_epochs:
            value = TimeBasedFeatureService.cosine_proxy(epoch.timestamp - first_timestamp)
            normalized_value = value
            features.append(normalized_value)

        return np.array(features)

    @staticmethod
    def build_circadian_model_from_raw(circadian_model, valid_epochs):
        features = []

        first_inactive_epoch = valid_epochs[0]
        first_value = np.interp(first_inactive_epoch.timestamp, circadian_model[:, 0], circadian_model[:, 1])

        for epoch in valid_epochs:
            time = epoch.timestamp
            value = np.interp(time, circadian_model[:, 0], circadian_model[:, 1])
            normalized_value = (value - first_value) / (np.amin((circadian_model[:, 1] - first_value)))

            if normalized_value < Constants.LOWER_BOUND:
                normalized_value = Constants.LOWER_BOUND

            features.append([normalized_value])

        feature_array = np.array(features)

        return feature_array

In [19]:
class FeatureBuilder(object):

    @staticmethod
    def build(subject_id):
        if Constants.VERBOSE:
            print("Getting valid epochs...")
        valid_epochs = RawDataProcessor.get_valid_epochs(subject_id)

        if Constants.VERBOSE:
            print("Building features...")
        FeatureBuilder.build_labels(subject_id, valid_epochs)
        FeatureBuilder.build_from_time(subject_id, valid_epochs)

    @staticmethod
    def build_labels(subject_id, valid_epochs):
        psg_labels = PSGLabelService.build(subject_id, valid_epochs)
        PSGLabelService.write(subject_id, psg_labels)

    @staticmethod
    def build_from_time(subject_id, valid_epochs):

        # circadian_feature = TimeBasedFeatureService.build_circadian_model(subject_id, valid_epochs)
        cosine_feature = TimeBasedFeatureService.build_cosine(valid_epochs)
        time_feature = TimeBasedFeatureService.build_time(valid_epochs)

        # TimeBasedFeatureService.write_circadian_model(subject_id, circadian_feature)
        TimeBasedFeatureService.write_cosine(subject_id, cosine_feature)
        TimeBasedFeatureService.write_time(subject_id, time_feature)

In [22]:
subject_ids = get_all_subject_ids()

start_time = time.time()
for subject in subject_ids:
    print(str(subject) + " Cropped data로 변환 시작")
    crop_all(str(subject))
    
for subject in subject_ids:
    FeatureBuilder.build(str(subject))
end_time = time.time()
print("실행 시간 " + str((end_time - start_time) / 60) + "분 걸림")



3509524 Cropped data로 변환 시작
5132496 Cropped data로 변환 시작
1066528 Cropped data로 변환 시작
5498603 Cropped data로 변환 시작
2638030 Cropped data로 변환 시작
2598705 Cropped data로 변환 시작
5383425 Cropped data로 변환 시작
1455390 Cropped data로 변환 시작
4018081 Cropped data로 변환 시작
9961348 Cropped data로 변환 시작
1449548 Cropped data로 변환 시작
8258170 Cropped data로 변환 시작
781756 Cropped data로 변환 시작
9106476 Cropped data로 변환 시작
8686948 Cropped data로 변환 시작
8530312 Cropped data로 변환 시작
3997827 Cropped data로 변환 시작
4314139 Cropped data로 변환 시작
1818471 Cropped data로 변환 시작
4426783 Cropped data로 변환 시작
8173033 Cropped data로 변환 시작
7749105 Cropped data로 변환 시작
5797046 Cropped data로 변환 시작
759667 Cropped data로 변환 시작
8000685 Cropped data로 변환 시작
6220552 Cropped data로 변환 시작
844359 Cropped data로 변환 시작
9618981 Cropped data로 변환 시작
1360686 Cropped data로 변환 시작
46343 Cropped data로 변환 시작
8692923 Cropped data로 변환 시작
실행 시간 5.489970497290293분 걸림


In [26]:
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

#from source import utils


class DataPlotBuilder(object):
    @staticmethod
    def timestamp_to_string(ts):
        return time.strftime('%H:%M:%S', time.localtime(ts))

    @staticmethod
    def convert_labels_for_hypnogram(labels):
        processed_labels = np.array([])

        for epoch in labels:
            if epoch == -1:
                processed_labels = np.append(processed_labels, 0)
            elif epoch == 5:
                processed_labels = np.append(processed_labels, 1)
            else:
                processed_labels = np.append(processed_labels, -1 * epoch)

        return processed_labels

    @staticmethod
    def tidy_data_plot(x_min, x_max, dt, ax):
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.spines['left'].set_visible(True)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        xticks = np.arange(x_min, x_max, dt)
        plt.xticks(xticks)
        labels = []
        for xt in xticks:
            labels.append(DataPlotBuilder.timestamp_to_string(xt))
        ax.set_xticklabels(labels)
        plt.xlim(x_min, x_max)

    @staticmethod
    def make_data_demo(subject_id="16", snippet=False):
        motion_color = [0.3, 0.2, 0.8]
        psg_color = [0.1, 0.7, 0.1]
        font_size = 16
        font_name = "Arial"

        data_path = str(Constants.CROPPED_FILE_PATH) + '/'
        output_path = str(Constants.FIGURE_FILE_PATH) + '/'

        if snippet is False:
            fig = plt.figure(figsize=(10, 12))
        else:
            fig = plt.figure(figsize=(3, 12))

        num_v_plots = 5
        fig.patch.set_facecolor('white')

        if (os.path.isfile(data_path + subject_id + '_cleaned_motion.out') and os.path.isfile(data_path + subject_id + '_cleaned_psg.out') and os.stat(data_path + subject_id + '_cleaned_motion.out').st_size > 0):

            motion = np.genfromtxt(data_path + subject_id + '_cleaned_motion.out', delimiter=' ')
            scores = np.genfromtxt(data_path + subject_id + '_cleaned_psg.out', delimiter=' ')

            min_time = min(scores[:, 0])
            max_time = max(scores[:, 0])
            dt = 60 * 60

            sample_point_fraction = 0.92

            sample_point = sample_point_fraction * (max_time - min_time) + min_time
            window_size = 10
            if snippet:
                min_time = sample_point
                max_time = sample_point + window_size

            ax = plt.subplot(num_v_plots, 1, 1)
            ax.plot(motion[:, 0], motion[:, 1], color=motion_color)
            ax.plot(motion[:, 0], motion[:, 2], color=[0.4, 0.2, 0.7])
            ax.plot(motion[:, 0], motion[:, 3], color=[0.5, 0.2, 0.6])
            plt.ylabel('Motion (g)', fontsize=font_size, fontname=font_name)
            DataPlotBuilder.tidy_data_plot(min_time, max_time, dt, ax)

            if snippet:
                ax.spines['bottom'].set_visible(True)
                ax.spines['left'].set_visible(True)
                ax.spines['top'].set_visible(True)
                ax.spines['right'].set_visible(True)

                ax.yaxis.label.set_visible(False)

                inds = np.intersect1d(np.where(motion[:, 0] > sample_point)[0],
                                      np.where(motion[:, 0] <= sample_point + window_size)[0])
                y_min = np.amin(motion[inds, 1:3])
                plt.ylim(y_min - 0.005, y_min + 0.025)

                # Get rid of the ticks
                ax.set_xticks([])
                ax.yaxis.set_ticks_position("right")

                plt.ylabel('')
                plt.xlabel(str(window_size) + ' sec window', fontsize=font_size, fontname=font_name)
            else:
                y_min = -3.2
                y_max = 2.5
                plt.ylim(y_min, y_max)
                current_axis = plt.gca()
                current_axis.add_patch(
                    Rectangle((sample_point, y_min), window_size, y_max - y_min, alpha=0.7, facecolor="gray"))


            ax = plt.subplot(num_v_plots, 1, 2)
            relabeled_scores = DataPlotBuilder.convert_labels_for_hypnogram(scores[:, 1])
            ax.step(scores[:, 0], relabeled_scores, color=psg_color)
            plt.ylabel('Stage', fontsize=font_size, fontname=font_name)
            plt.xlabel('Time', fontsize=font_size, fontname=font_name)
            DataPlotBuilder.tidy_data_plot(min_time, max_time, dt, ax)
            ax.set_yticks([-4, -3, -2, -1, 0, 1])
            ax.set_yticklabels(['N4', 'N3', 'N2', 'N1', 'Wake', 'REM'])

            if snippet:
                plt.axis('off')
                plt.ylim(5, 5)
            else:
                plt.ylim(-5, 2)

            if not snippet:
                plt.savefig(output_path + 'data_validation_' + subject_id + '.png', bbox_inches='tight', pad_inches=0.1, dpi=300)
            else:
                plt.savefig(output_path + 'data_validation_zoom_' + subject_id + '.png', bbox_inches='tight',pad_inches=0.1, dpi=300)
            plt.close()

In [27]:
subject_ids = get_all_subject_ids()
for subject in subject_ids:
    DataPlotBuilder.make_data_demo(subject, False)